# The Classification Code

## Should be run after the weights determined from the Transformer autoencoder.

Author: Kara Ponder (SLAC)

In [ ]:
import tensorflow as tf
import numpy as np

from transformer import Encoder, Decoder, create_decoder_masks

In [ ]:
# Set parameters
d_model = 128 # input vector must have length d_model
target_vocab_size = 6  # possible results to choose from

lc_length = 50 + 1 # light curve length
input_vocab_size = lc_length

## hyperparameters:
num_layers = 8
dropout_rate = 0.0
dff = 64 # hidden layer size of the feed forward network, needs to be larger than 24
num_heads = 8 # d_model % num_heads == 0 

# LC stuff
N = 500 # number of objects
N_days = 100 + 1
Nf = 6 # number of filters
num_class = 4 # set to 4 for the test data. Will be more for PLAsTiCC

batch_size=64

This function does the classification

In [ ]:
def classify_ffn(nclass, dff, rate=0.0):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(dff, activation='relu')) 
    model.add(tf.keras.layers.Dropout(rate))
    model.add(tf.keras.layers.Dense(dff, activation='relu'))
    model.add(tf.keras.layers.GlobalMaxPool1D())
    model.add(tf.keras.layers.Dense(nclass, activation='softmax'))
    return model

Initialize transformer model in order to extract the encoder layers that will be used to encode before classification

In [ ]:
encoder = Encoder(num_layers, d_model, num_heads, dff,
                       lc_length, dropout_rate, embed=True)

decoder = Decoder(num_layers, d_model, num_heads, dff,
                       lc_length, dropout_rate, embed=True)

final_layer = tf.keras.layers.Dense(target_vocab_size)

inp = tf.keras.layers.Input(shape=(None,6))#shape=(None,None))#
target = tf.keras.layers.Input(shape=(None,6))#shape=(None,None))#

x = encoder(inp)
x = decoder(target, x, mask=create_decoder_masks(inp, target))

x = final_layer(x)

model = tf.keras.models.Model(inputs=[inp, target], outputs=x)

Load the previous weights

In [ ]:
model.load_weights('TRANSFORMER_WEIGHTS.h5')

`extras` corresponds to the MIN/MAX normalization constants per objects that are saved when preprocessing the lightcurves.

In [ ]:
extras = tf.keras.layers.Input(shape=(None, 2), dtype=tf.float32)

Define the classifier by taking the Transformer encoder layer and not training those weights. 

In [ ]:
cl_inp = tf.keras.layers.Input(shape=(None,6))

classify_encoder = Encoder(num_layers, d_model, num_heads, dff,
                       lc_length, dropout_rate, embed=True)
classify_encoder(cl_inp)
classify_encoder.set_weights(model.get_layer(name='encoder').get_weights())

classify_encoder.trainable = False

class_ffn = classify_ffn(num_class, dff, rate=dropout_rate)

cl_x = classify_encoder(inp)

# Add in normalization constants
cl_x = tf.keras.layers.Concatenate(axis=-1)([cl_x, extras])
cl_x = class_ffn(cl_x)

aeclass = tf.keras.models.Model(inputs=[inp, extras], outputs=cl_x)

Define optimizer and compile the AEClassifier

In [ ]:
optimizer = tf.keras.optimizers.Adam(0.0001)

loss_object = tf.keras.losses.CategoricalCrossentropy(

train_loss = tf.keras.metrics.Mean(name='train_loss')

train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

aeclass.compile(loss=loss_object,
                optimizer=optimizer, 
                metrics=['accuracy'])

Load simplified data for illustration

In [ ]:
label = np.load('label.npy')
lc_data = np.load('lc_data.npy')
norm_data = np.loadtxt('min_max.txt')

labels = tf.keras.utils.to_categorical(label, num_classes=num_class, dtype="float64")


In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((lc_data, labels, norm_data))
batch_ds = dataset.batch(batch_size)

Define number of batch steps and 

In [ ]:
num_batches = 0
for (batch, _) in enumerate(batch_ds):
    num_batches = batch

def generator(data_set):
    while True:
        for in_batch, tar_batch, norm_batch in data_set:
            yield ( [in_batch, norm_batch] , tar_batch)

Fit for the AEClassifier weights

In [ ]:
history = aeclass.fit(x = generator(batch_ds),
                    #validation_data = generator(batch_ds_VALID),
                    epochs=15,
                    steps_per_epoch = num_batches,
                    #validation_steps = num_batches_VALID,
                    )

In [ ]:
#aeclass.save_weights('save_aeclass_weights.h5')